In [ ]:
import random
import pandas
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt

# set up the RNA nucleotide comp

In [ ]:
print(pandas.__version__)
print(np.__version__)

In [ ]:
import random

RNAntscon = {"A":25, "C":25, "T":25, "G":25}
#randomList = 

# Build in-silico pieces of DNA and then transcribe them with a certain stop rate based on the ratio of biotinCTP to CTP

In [ ]:
def generaterandomRNA(lenofpotentialRNA):
    keys = list(RNAntscon.keys())
    values = [RNAntscon[key] for key in keys]
    RNAlist = random.choices(keys, values, k=lenofpotentialRNA)
    return "".join(RNAlist)

In [ ]:
def testgenerate():
    print (generaterandomRNA(5))

In [ ]:
def transcription_biotinincorp(RNA,RNApolIIpos, runonbiotinC, runonC, naturalC):
    #print (RNA)
    stoplist = ["T", "F"]
    currenttranscript=RNA[:RNApolIIpos]
    currenttranscript = currenttranscript.split()
    restofRNA = RNA[RNApolIIpos:]
    #print ("current RNA", currenttranscript)
    #print ("potential RNA", restofRNA)
    for nt in restofRNA:
        currenttranscript.append(nt)
        if nt=="C":
            doIstop = random.choices(stoplist, weights=(runonbiotinC, runonC+naturalC))
            if doIstop[0]=="T":
                currenttranscript = "".join(currenttranscript)
                #print ("final RNA", currenttranscript)
                return currenttranscript
    currenttranscript = "".join(currenttranscript)
    #print ("final RNA", currenttranscript)
    return currenttranscript


def lendist(lenofpotentialRNA, RNApolIIpos, repeats,runonbiotinC, runonC, naturalC):
    stoppedlens = []
    for i in range(repeats):
        potentialtranscript = generaterandomRNA(lenofpotentialRNA)
        finalbiotinalatedRNA = transcription_biotinincorp(potentialtranscript,RNApolIIpos , runonbiotinC, runonC, naturalC)
        stoppedlens.append(len(finalbiotinalatedRNA))
    return stoppedlens

In [ ]:
def testlendist():
    lenofpotentialRNA=500
    RNApolIIpos=5
    repeats = 10
    runonbiotinC=2
    runonC=0
    naturalC=0
    stoppedlens = lendist(lenofpotentialRNA=lenofpotentialRNA, RNApolIIpos=RNApolIIpos, repeats=repeats, runonbiotinC=runonbiotinC, runonC=runonC, naturalC=naturalC)
    print (stoppedlens)

In [ ]:
def differentbiotincon(RNApolIIpos,lenofpotentialRNA,repeats, naturalC=0):
    experiments = [[2,0],[3,5],[2,8]]
    results = {}
    for Cconpair in experiments:
        runonbiotinC, runonC = Cconpair
        name = str(Cconpair)
        stoppedlens = lendist(lenofpotentialRNA=lenofpotentialRNA, RNApolIIpos=RNApolIIpos, repeats=repeats, runonbiotinC=runonbiotinC, runonC=runonC, naturalC=naturalC)
        results[name]=stoppedlens
    df = pandas.DataFrame.from_dict(results)
    df = pandas.melt(df)
    df = df.rename({"value":"final_length_of_transcript", "variable": "Biotin_consentration"}, axis=1)
    df["lenofpotentialRNA"]=lenofpotentialRNA
    df["RNApolIIpos"]=RNApolIIpos
    df["filtered"] = "N"
    return df
        

# How long is the biotinlated transcript when pol II is at 5 nt from the start site. 

In [ ]:
biotinRNAscreated = differentbiotincon(RNApolIIpos=5,lenofpotentialRNA=500,repeats = 100)
fig = px.violin(biotinRNAscreated, y="final_length_of_transcript", x="Biotin_consentration", color="RNApolIIpos", box=True, points="all",
          hover_data=biotinRNAscreated.columns)
fig.show()


In [ ]:
biotinRNAscreated = differentbiotincon(RNApolIIpos=20,lenofpotentialRNA=500,repeats = 100)
fig = px.violin(biotinRNAscreated, y="final_length_of_transcript", x="Biotin_consentration", color="RNApolIIpos", box=True, points="all",
          hover_data=biotinRNAscreated.columns)
fig.show()

# Ampure bead slection

## If the ampure bead is at 0.9x, it gets rid of most things at 100. The cDNA length at that point is sum of the two adapters and the RNA length. If I remember right the sum of the two adapters as ~63. So basiacaly any RNA under 30 nt in length will be gone. 


In [ ]:

def filtertoshort(biotinRNAscreated, toshort=1):
    filterlist = []
    for value in biotinRNAscreated['final_length_of_transcript']:
        sizefilter=round((np.random.exponential((1/(((toshort)/10))),1))[0]+15)
        filterlist.append(sizefilter)
    biotinRNAscreated['sizeIllfilter'] = np.array(filterlist)
    biotinRNAscreated = biotinRNAscreated[biotinRNAscreated["final_length_of_transcript"]>biotinRNAscreated["sizeIllfilter"]]
    biotinRNAscreated["filtered"] = "Ampure"+str(toshort) 
    print ("columnsafterfileter", biotinRNAscreated.shape)
    return biotinRNAscreated

In [ ]:
toshort=1
biotinRNAscreated = differentbiotincon(RNApolIIpos=20,lenofpotentialRNA=500,repeats = 100)
biotinRNAscreated_removeshort = filtertoshort(biotinRNAscreated, toshort=toshort)
filterandnotmanypolposlistdf = pandas.concat([biotinRNAscreated, biotinRNAscreated_removeshort])
fig = px.violin(filterandnotmanypolposlistdf, y="final_length_of_transcript", x="Biotin_consentration", color="filtered", box=True, points="all",
          hover_data=biotinRNAscreated.columns)
fig.show()



# In reality the position of poll II on any transcript is a mixture of all the RNA pol II in the cell. I need to model typical Pol II position in the cell. 

In [ ]:
def modelpolIIposloading(mu, sigma,intnumpol):
    polIIpos = np.random.randn(intnumpol) * sigma + mu
    return polIIpos

def modelloadandinit(mu, sigma,variablethatcontrolshowlonginitRNAsare, intnumpol):
    polIIloadpostions = list(modelpolIIposloading(mu, sigma,intnumpol))
    endinits = []
    for pos in polIIloadpostions:
        newpos = np.random.exponential(variablethatcontrolshowlonginitRNAsare,1)
        newpos = newpos[0]+pos
        endinits.append(newpos)
    return endinits, polIIloadpostions, ["loadinit" for num in polIIloadpostions]

def modelloadandelongate(mu, end,elongnumpol):
    polIIloadpostions = list(modelpolIIposloading(mu, sigma,elongnumpol))
    endinits = []
    for pos in polIIloadpostions:
        newpos = np.random.uniform(pos,end,1)
        newpos = newpos[0]
        endinits.append(newpos)
    return endinits, polIIloadpostions, ["loadelong" for num in polIIloadpostions]


def modelpolloadinginitelong(oribins, mu=0.0, sigma=30.0,variablethatcontrolshowlonginitRNAsare=1,ntnumpol=100,end=500, elongnumpol=10, graph="Y"):
    mu = mu+offset
    end = end+offset
    print (mu, end)
    modelofinit = modelloadandinit(mu, sigma,variablethatcontrolshowlonginitRNAsare,intnumpol)
    modelofelong = modelloadandelongate(mu,end, elongnumpol)
    producedby = modelofinit[2]+modelofelong[2]
    actuallmupos = modelofinit[1]+modelofelong[1]
    actuallmupos = [round(num) for num in actuallmupos]
    polIIpos = modelofinit[0]+modelofelong[0]
    wholepolIIpos = [round(num) for num in polIIpos]
    wholepolIIpos_removeoffset = [num-offset for num in wholepolIIpos]
    counts, bins = np.histogram(wholepolIIpos_removeoffset, bins=oribins)
    bins = 0.5 * (bins[:-1] + bins[1:])
    if graph=="Y":
        fig = px.bar(x=bins, y=counts, labels={'x':'position', 'y':'# pol'})
        fig.show()
    allpolexp = list(zip(actuallmupos, wholepolIIpos, producedby))
    df = pandas.DataFrame(allpolexp, columns=["cap pos","pol position", "produced by"])
    df["len of RNA"]=df["pol position"]-df["cap pos"]
    return df


In [ ]:
offset = 1000
oribins=list(range(-100,100,1))
mu=0
end=500
sigma=1
variablethatcontrolshowlonginitRNAsare=10 #this is not 1:1. A value of 1 give a distibution of initating RNAs that are less than 10, value of #5 up to 50
intnumpol=1000000
elongnumpol=100000
df_modeled_pol = modelpolloadinginitelong(oribins, mu, sigma,variablethatcontrolshowlonginitRNAsare,intnumpol,end, elongnumpol)

### How long are the RNAs in this model of polII

#### initating RNAs

In [ ]:


initdf = df_modeled_pol[df_modeled_pol["produced by"]=="loadinit"]
fig = px.violin(initdf, x="cap pos", y="len of RNA", title="Elongating RNAs")
fig.show()

In [ ]:
elongdf = df_modeled_pol[df_modeled_pol["produced by"]=="loadelong"]
fig = px.violin(elongdf, x="cap pos", y="len of RNA", title="Elongating RNAs")
fig.show()

# simulate the experiment

## code for simulation

In [ ]:

def simulatetranscriptionrunon(row, runonbiotinC, runonC, naturalC):
    stoplist = ["T", "F"]
    cap_pos = int(row["cap_pos"])
    potentialRNA = row["simulatedDNA"]
    len_of_RNA_pretranscription = row["len_of_RNA_pretranscription"]
    RNApolIIpos = int(row["polIIpos_prior"])
    currenttranscript=potentialRNA[int(cap_pos):int(RNApolIIpos)]
    currenttranscript = currenttranscript.split()
    restofRNA = potentialRNA[RNApolIIpos:]
    for i, nt in enumerate(restofRNA):
        currenttranscript.append(nt)
        if nt=="C":
            doIstop = random.choices(stoplist, weights=(runonbiotinC, runonC+naturalC))
            if doIstop[0]=="T":
                currenttranscript = "".join(currenttranscript)
                len_of_RNA_posttranscription=len(currenttranscript)
                polIIpos_afterrunon = len_of_RNA_posttranscription+cap_pos
                nt_transcripted = len_of_RNA_posttranscription-len_of_RNA_pretranscription
                #print ("final RNA", currenttranscript)
                return currenttranscript, polIIpos_afterrunon, len_of_RNA_posttranscription, nt_transcripted
    currenttranscript = "".join(currenttranscript)
    # If we never stop, we should throw this read out and try again
    # Currently, we can accomplish this by just assigning a read length of 1, so it doesn't even show up no matter what filter we use
    #print ("final RNA", currenttranscript)
    len_of_RNA_posttranscription = 1
    nt_transcripted = 1
    polIIpos_afterrunon = 1
    return currenttranscript, polIIpos_afterrunon, len_of_RNA_posttranscription, nt_transcripted


def getreadstart(row, maxreadlength):
    readstop = int(row["polIIpos_prior_withoutoffset"])
    readstart = int(int(readstop)-int(maxreadlength))
    if readstart<row["cappos_withoutoffset"]:
        readstart = row["cappos_withoutoffset"]
    return readstart
    
def runtranscription_onpolIImodel(mu, sigma,variablethatcontrolshowlonginitRNAsare,end, intnumpol, elongnumpol,lenofpotentialRNA, toshort, experimentsdf, maxreadlength):
    print ("modeling pol II")
    dfs = []
    df_modeled_pol = modelpolloadinginitelong(oribins, mu, sigma,variablethatcontrolshowlonginitRNAsare, intnumpol,end, elongnumpol, graph="N")
    df_modeled_pol = df_modeled_pol.rename({"cap pos":"cap_pos", "pol position":"polIIpos_prior", "produced by":"pol_produced_by", "len of RNA":"len_of_RNA_pretranscription"}, axis=1)
    df_modeled_pol["cappos_withoutoffset"]= df_modeled_pol["cap_pos"]-offset
    df_modeled_pol["polIIpos_prior_withoutoffset"]= df_modeled_pol["polIIpos_prior"]-offset
    df_modeled_pol["model_mu_withoutoffset"]=mu
    df_modeled_pol["mu"]=mu+1000
    df_modeled_pol["simulatedDNA"]= generaterandomRNA(lenofpotentialRNA+offset)
    df_modeled_pol["lensimulatedDNA"] = df_modeled_pol["simulatedDNA"].apply(len)
    for index, row in experimentsdf.iterrows():
            runonbiotinC = row["runonbiotinC"]
            runonC = row["runonC"]
            naturalC = row["naturalC"]
            name = row["name"]
            print (name)
            df_modeled_pol_exp =  df_modeled_pol.apply(lambda row : simulatetranscriptionrunon(row, runonbiotinC, runonC, naturalC), axis = 1, result_type='expand')
            df_modeled_pol_exp = df_modeled_pol_exp.rename({0:"biotinalated_transcript", 1:"polIIpos_afterrunon", 2:"final_length_of_transcript", 3:"nt_transcripted"}, axis="columns")
            df_modeled_pol_exp = pandas.concat([df_modeled_pol, df_modeled_pol_exp], axis='columns')
            df_modeled_pol_exp["biotin_con"]=name
            df_modeled_pol_exp["polIIpos_afterrunon_withoutoffset"]=df_modeled_pol_exp["polIIpos_afterrunon"]-offset
            df_modeled_pol_exp["filtered"] = "N"
            dfs.append(df_modeled_pol_exp)
            df_modeled_pol_exp_filtered = filtertoshort(df_modeled_pol_exp, toshort=toshort)
            dfs.append(df_modeled_pol_exp_filtered)
    df_modeled_pol_run_on = pandas.concat(dfs)
    df_modeled_pol_run_on["readstart_withoutoffset"]=df_modeled_pol_run_on.apply(lambda row : getreadstart(row, maxreadlength), axis = 1)
    return df_modeled_pol_run_on



def drawsammetagene(df_modeled_pol_run_on):
    dfs = []
    for experiment in df_modeled_pol_run_on["biotin_con"].unique():
        for filterlevel in df_modeled_pol_run_on["filtered"].unique():
            print (experiment, filterlevel)
            all_read_positions = []
            thismodel = df_modeled_pol_run_on[df_modeled_pol_run_on["biotin_con"]==experiment]
            thismodel = thismodel[thismodel["filtered"]==filterlevel]
            read_positions = []
            readsdf = thismodel[["readstart_withoutoffset", "polIIpos_afterrunon_withoutoffset"]]
            readsdf = readsdf.rename({"readstart_withoutoffset":"readstart", "polIIpos_afterrunon_withoutoffset": "readstop"}, axis="columns")
            for index, row in readsdf.iterrows():
                read_positions = list(range(int(row["readstart"]),int(row["readstop"])))
                all_read_positions = all_read_positions+read_positions
            counter = {}
            for position in set(all_read_positions):
                counter[position]=all_read_positions.count(position)
            poscountdf = pandas.DataFrame.from_dict(counter, orient='index')
            poscountdf["position"]=poscountdf.index
            poscountdf["biotin_con"] = experiment
            poscountdf["filtered"]=filterlevel
            poscountdf = poscountdf.rename({0:"readcount"}, axis="columns")
            dfs.append(poscountdf)
    df_modeled_pol_run_on_metagene = pandas.concat(dfs)
    df_modeled_pol_run_on_metagene = df_modeled_pol_run_on_metagene.sort_values(by=["biotin_con", "filtered", "position"])
    print (df_modeled_pol_run_on_metagene.shape)
    print (df_modeled_pol_run_on_metagene.columns)
    fig = px.line(df_modeled_pol_run_on_metagene, x="position", y="readcount",color="biotin_con", line_dash="filtered", title='simulated metagene')
    #fig.update_xaxes(range=[mu-plusandminus,mu+plusandminus])
    fig.show()
    return df_modeled_pol_run_on_metagene

def drawsammetagene_3prime(df_modeled_pol_run_on):
    dfs = []
    for experiment in df_modeled_pol_run_on["biotin_con"].unique():
        for filterlevel in df_modeled_pol_run_on["filtered"].unique():
            print (experiment, filterlevel)
            all_read_positions = []
            thismodel = df_modeled_pol_run_on[df_modeled_pol_run_on["biotin_con"]==experiment]
            thismodel = thismodel[thismodel["filtered"]==filterlevel]
            read_positions = []
            readsdf = thismodel[["readstart_withoutoffset", "polIIpos_afterrunon_withoutoffset"]]
            readsdf = readsdf.rename({"readstart_withoutoffset":"readstart", "polIIpos_afterrunon_withoutoffset": "readstop"}, axis="columns")
            for index, row in readsdf.iterrows():
                read_positions = [row["readstop"]]
                all_read_positions = all_read_positions+read_positions
            counter = {}
            for position in set(all_read_positions):
                counter[position]=all_read_positions.count(position)
            poscountdf = pandas.DataFrame.from_dict(counter, orient='index')
            poscountdf["position"]=poscountdf.index
            poscountdf["biotin_con"] = experiment
            poscountdf["filtered"]=filterlevel
            poscountdf = poscountdf.rename({0:"readcount"}, axis="columns")
            dfs.append(poscountdf)
    df_modeled_pol_run_on_metagene = pandas.concat(dfs)
    df_modeled_pol_run_on_metagene = df_modeled_pol_run_on_metagene.sort_values(by=["biotin_con", "filtered", "position"])
    print (df_modeled_pol_run_on_metagene.shape)
    print (df_modeled_pol_run_on_metagene.columns)
    fig = px.line(df_modeled_pol_run_on_metagene, x="position", y="readcount",color="biotin_con", line_dash="filtered", title='simulated metagene')
    #fig.update_xaxes(range=[mu-plusandminus,mu+plusandminus])
    fig.show()
    return df_modeled_pol_run_on_metagene




## set up the biotin concentration dataframe

In [ ]:
experimentsdf = pandas.DataFrame.from_records([[10,1,0],[1,10,0],[1,1,0]], columns = ["runonbiotinC", "runonC", "naturalC"])
experimentsdf["name"]= "biotin"+experimentsdf["runonbiotinC"].astype(str)+"_addedcold"+experimentsdf["runonC"].astype(str)+"_naturalC"+experimentsdf["naturalC"].astype(str)
experimentsdf


## run the in silico experiment 

In [ ]:
offset = 1000 #I use a offset so that none of the cap positions are negative (since mu is 0 and there is a sigma around it)
#controling pol II
variablethatcontrolshowlonginitRNAsare=10 #this is not 1:1. A value of 1 give a distibution of initating RNAs that are less than 10, value of #5 up to 50
mu=0#this is the mu position
end=10000 
lenofpotentialRNA=end+100 #lenofpotentialRNA number is for simulating the DNA only. The end is the end of transcription. I have to simulate a piece of DNA slightly longer so that I have room if the cap is not at mu.
sigma=3
intnumpol=20000
elongnumpol=15000
#"Ampure" size selection, i.e., higher numbers mean smaller reads are retained. This is NOT exactly the same as Ampure bead ratios
toshort=2.5
#controling how long the reads are
maxreadlength=150
df_modeled_pol_run_on_metagene = runtranscription_onpolIImodel(mu, sigma,variablethatcontrolshowlonginitRNAsare,end, intnumpol, elongnumpol,lenofpotentialRNA, toshort,experimentsdf, maxreadlength)


## graph the metagene that would be made by the simulation

In [ ]:
readsdf = drawsammetagene(df_modeled_pol_run_on_metagene)


## graph the 3' end of read metagene that would be made by the simulation

In [ ]:
readsdf = drawsammetagene_3prime(df_modeled_pol_run_on_metagene)


In [ ]:
df_modeled_pol_run_on_metagene.to_csv("/Users/samuelhunter/library_potpourri/reanalysis/sim_df_ampure250.csv")